# Gretel Relational
Synthetics and Transforms for relational data.

## Quickstart

In [ ]:
from gretel_client import configure_session

configure_session(api_key="prompt", cache="yes", validate=True)

In [ ]:
# End-to-end synthetics example

from gretel_trainer.relational import MultiTable, sqlite_conn


!curl -o "ecom_xf.db" "https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecom_xf.db"


connector = sqlite_conn("ecom_xf.db")
relational_data = connector.extract()

mt = MultiTable(relational_data)
mt.train_synthetics(config="synthetics/amplify")
mt.generate()

connector.save(mt.synthetic_output_tables, prefix="synthetic_")

## Detailed walkthrough

### Set up source relational data

In [ ]:
# Display the schema of our demo database

from IPython.display import Image

Image("https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecommerce_db.png", width=600, height=600)

In [ ]:
# Download the demo database

!curl -o "ecom_xf.db" "https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecom_xf.db"

The core Python object capturing source relational data and metadata is named `RelationalData`.
It can be created automatically using a `Connector`, or it can be created manually.


In [ ]:
# Connect to SQLite database and extract relational data

from gretel_trainer.relational import sqlite_conn

ecommerce_db_path = "ecom_xf.db"

sqlite = sqlite_conn(path=ecommerce_db_path)
relational_data = sqlite.extract()

In [ ]:
# Alternatively, manually define relational data

from gretel_trainer.relational import RelationalData
import pandas as pd

csv_dir = "/path/to/extracted_csvs"

tables = [
    ("events", "id"),
    ("users", "id"),
    ("distribution_center", "id"),
    ("products", "id"),
    ("inventory_items", "id"),
    ("order_items", "id"),
]

foreign_keys = [
    {
        "table": "events",
        "constrained_columns": ["user_id"],
        "referred_table": "users",
        "referred_columns": ["id"],
    },
    {
        "table": "order_items",
        "constrained_columns": ["user_id"],
        "referred_table": "users",
        "referred_columns": ["id"],
    },
    {
        "table": "order_items",
        "constrained_columns": ["inventory_item_id"],
        "referred_table": "inventory_items",
        "referred_columns": ["id"],
    },
    {
        "table": "inventory_items",
        "constrained_columns": ["product_id"],
        "referred_table": "products",
        "referred_columns": ["id"],
    },
    {
        "table": "inventory_items",
        "constrained_columns": ["product_distribution_center_id"],
        "referred_table": "distribution_center",
        "referred_columns": ["id"],
    },
    {
        "table": "products",
        "constrained_columns": ["distribution_center_id"],
        "referred_table": "distribution_center",
        "referred_columns": ["id"],
    },
]

rel_data = RelationalData()

for table, pk in tables:
    rel_data.add_table(name=table, primary_key=pk, data=pd.read_csv(f"{csv_dir}/{table}.csv"))

for foreign_key in foreign_keys:
    rel_data.add_foreign_key_constraint(**foreign_key)

Regardless of how it was created, a `RelationalData` instance can be modified after creation if necessary. In addition to the methods in the manual example above, you can modify source table data, change primary keys, and remove foreign keys.

In [ ]:
# We don't actually want to make these changes; they just serve as examples

# Overwrite source data with a different dataframe
# rel_data.update_table_data(table="users", data=pd.read_csv("alt_users.csv"))

# Change which column (if any) is designated as the primary key
# rel_data.set_primary_key(table="distribution_center", primary_key="name")
# rel_data.set_primary_key(table="order_items", primary_key=None)

# Remove a foreign key relationship
# rel_data.remove_foreign_key("inventory_items.product_distribution_center_id")

### Operate on the source data

The `MultiTable` class is the interface to working with relational data. It requires a `RelationalData` instance. Several other options can be configured; the defaults are shown below as comments.

In [ ]:
from gretel_trainer.relational import MultiTable

multitable = MultiTable(
    relational_data,
    # project_display_name="multi-table",
    # strategy="independent",
    # refresh_interval=60,
)

#### Classify

Run Gretel Classify on all tables to identify PII.

By default, Relational Classify will provide results for the first 100 rows in each table. To process the entire table, set `all_rows=True`.

In [ ]:
import yaml

config_yaml = """
schema_version: "1.0"
name: "classify-default"
models:
  - classify:
      data_source: "_"
      labels:
        - person_name
        - credit_card_number
        - phone_number
        - us_social_security_number
        - email_address
        - location
        - acme/*
        
label_predictors:
  namespace: acme
  regex:
    user_id:
      patterns:
        - score: high
          regex: ^user_[\d]{5}$
"""
config = yaml.safe_load(config_yaml)

multitable.classify(config)

# Run classify on all rows
# multitable.classify(config, all_rows=True)

#### Transforms

Train Gretel Transforms models by providing a transforms model config. By default this config will be applied to all tables. You can limit the tables being transformed via the optional `only` (tables to include) or `ignore` (tables to exclude) arguments.

In [ ]:
config = "https://raw.githubusercontent.com/gretelai/gdpr-helpers/main/src/config/transforms_config.yaml"

multitable.train_transforms(config)

# Optionally limit which tables are trained for transforms via `only` (included) or `ignore` (excluded).
# Given our example data, the two calls below will lead to the same tables getting trained, just specified different ways.
#
# multitable.train_transforms(config, ignore={"distribution_center", "products"})
# multitable.train_transforms(config, only={"users", "events", "inventory_items", "order_items"})

Run transforms to get transformed output. Each call to `run_transforms` is assigned (or supplied) a unique identifier; look for the transformed output tables in a subdirectory matching that identifier name in the working directory. An archive file containing all runs' outputs is also uploaded to the Gretel project as a project artifact, visible in the Data Sources tab in the Console.

By default, `run_transforms` operates on the original source data for all tables with successfully completed transforms models.

You can optionally run other data through transforms by passing it in as Pandas DataFrames to the optional `data` argument. In this case, only the provided tables will be transformed (not _all_ tables as in the default, no-`data`-argument case).

If you intend to train synthetic models on the transformed output instead of the original source data, add the argument `in_place=True`. **This will modify the data in the `RelationalData` instance.**

In [ ]:
multitable.run_transforms()

# Provide a specific identifier for the run (default is `transforms_{timestamp}`)
# multitable.run_transforms(identifier="my-transforms-run")

# Overwrite source data so that future synthetics actions consider the transformed output as the source
# multitable.run_transforms(in_place=True)

# Run other data through the trained transforms models
# multitable.run_transforms(data={"events": some_other_events_dataframe})

In [ ]:
# Compare original to transformed

print(multitable.relational_data.get_table_data("users").head(5))
print(multitable.transform_output_tables["users"].head(5))

#### Synthetics

Start by training models for synthetics. By default, a synthetics model will be trained for every table in the `RelationalData`. However, this scope can be reduced to a subset of tables using the optional `only` (tables to include) or `ignore` (tables to exclude) arguments. This can be particularly useful if certain tables contain static reference data that should not be synthesized.

In [ ]:
# Train synthetic models for all tables

multitable.train_synthetics(config="synthetics/amplify")

# Optionally limit which tables are trained for synthetics via `only` (included) or `ignore` (excluded).
# Given our example data, the two calls below will lead to the same tables getting trained, just specified different ways.
#
# multitable.train_synthetics(config="synthetics/amplify", ignore={"distribution_center", "products"})
# multitable.train_synthetics(config="synthetics/amplify", only={"users", "events", "inventory_items", "order_items"})

When training is complete, you'll find a number of artifacts in your working directory, including the CSVs on which models were trained (`synthetics_train_{table}.csv`) and evaluation reports (`synthetics_[type]_evaluation_{table}.[html|json]`).

You can also view some evaluation metrics at this point. (We'll expand upon them after generating synthetic data.)

In [ ]:
multitable.evaluations

Each synthetic data generation run is assigned (or supplied) a unique identifier. Look for a subdirectory with this identifier name in the working directory to find all synthetic outputs, including data and reports. An archive file containing all runs' outputs is also uploaded to the Gretel project as a project artifact, visible in the Data Sources tab in the Console.

When you generate synthetic data, you can optionally change the amount of data to generate via `record_size_ratio`.

In [ ]:
# Generate synthetic data

multitable.generate()

# Provide a specific identifier for the run (default is `synthetics_{timestamp}`)
# multitable.generate(identifier="my-synthetics-run")

# Generate twice as much synthetic data
# multitable.generate(record_size_ratio=2.0)

In [ ]:
# Compare original to synthetic data

print(multitable.relational_data.get_table_data("users").head(5))
print(multitable.synthetic_output_tables["users"].head(5))

If we take another look at our evaluations, we'll see additional metrics are available.

In [ ]:
multitable.evaluations

We've also automatically generated a full relational report summarizing and explaining all this information. Look for `relational_report.html` in the generate run subdirectory.

In [ ]:
import IPython
from smart_open import open

report_path = str(multitable._working_dir / multitable._synthetics_run.identifier / "relational_report.html")

IPython.display.HTML(data=open(report_path).read())

The synthetic data is automatically written to the working directory in CSV format as `synth_{table}.csv`. You can optionally use a `Connector` to write the synthetic data to a database. (If you're writing back to the same database as your source, pass a `prefix: str` argument to the `save` method to avoid overwriting your source tables!)

In [ ]:
# Write output data to a new SQLite database

from gretel_trainer.relational import sqlite_conn

synthetic_db_path = "out.db"

synthetic_db_conn = sqlite_conn(synthetic_db_path)
synthetic_db_conn.save(multitable.synthetic_output_tables)

### Postgres demo via Docker

In [ ]:
# Start up a postgres container with docker

!docker run --rm -d --name multitable_pgdemo -e POSTGRES_PASSWORD=password -p 5432:5432 postgres

In [ ]:
# Write synthetic tables to the Postgres db

from gretel_trainer.relational import postgres_conn

out_db = postgres_conn("postgres", "password", "localhost", 5432)
out_db.save(multitable.synthetic_output_tables)


In [ ]:
# Inspect the postgres database

!docker exec multitable_pgdemo psql -U postgres -c "\dt"
!docker exec multitable_pgdemo psql -U postgres -c "select * from users limit 5;"

In [ ]:
# Tear down the docker container

!docker stop multitable_pgdemo